# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图片呈现了一处风景优美的自然景观，场景中的一条木栈道穿越着郁郁葱葱的草地。木栈道指引着视线深入景深，引发人对未知的好奇及探索欲。草地两侧种植着各种树木和灌木，增添了多样性和层次感。\n\n在这个景象中，天空呈现出丰富的蓝色和白色，云朵轻盈飘逸，映衬着下方的绿色大地。这种对比和色彩的鲜明对比给人一种平静和自然的感觉，适合于心灵的放松和沉思。整体上，这是一个非常适合徒步和自然观赏的地点。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图片呈现了一处风景优美的自然景观，场景中的一条木栈道穿越着郁郁葱葱的草地。木栈道指引着视线深入景深，引发人对未知的好奇及探索欲。草地两侧种植着各种树木和灌木，增添了多样性和层次感。\n\n在这个景象中，天空呈现出丰富的蓝色和白色，云朵轻盈飘逸，映衬着下方的绿色大地。这种对比和色彩的鲜明对比给人一种平静和自然的感觉，适合于心灵的放松和沉思。整体上，这是一个非常适合徒步和自然观赏的地点。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默搞笑的图片，展示了两种经过想象修改的狗狗形象。左边是一只身体像健美运动员一样肌肉发达的柴犬，右边则是一只看起来比较普通、带有些许忧郁表情的柴犬。图片上方的文字“16岁的我”和“工作后的我”暗示了这两幅图的对比，可能寓意着年轻时充满活力和希望，而工作后可能感到疲惫或失落。这种对比通常用来表达对现实生活中成长和职业生涯中变化的幽默看法。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [12]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="如何解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [10]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图表是从1980年到2020年各国国内生产总值（GDP）的比较。图中显示了四个国家的GDP变化趋势：美国（蓝色线），中国（红色线），日本（紫色线），和德国（绿色线）。

从图表中可以看出以下几点：
1. 美国的GDP在这40年期间持续增长，尤其从2000年后增长幅度加大，呈现出稳定的上升趋势。
2. 中国的GDP增长最为显著，特别是从2000年开始，增长速度大幅上升，到2020年接近美国的GDP。
3. 日本的GDP在1990年代早期达到高峰，之后增长放缓，甚至有几年时间出现下滑。
4. 德国的GDP增长相对平稳，没有出现很大的波动，总体呈稳步上升趋势。

这些数据显示了不同国家的经济发展轨迹和宏观经济政策的效果，以及它们在全球经济中的地位变化。特别是中国的迅猛增长，改变了全球经济的格局。


#### 使用 GPT-4V 识别手写体笔记

![](./images/note.jpg)

In [15]:
content = query_base64_image_description("./images/note.jpg", "作为一名中学生，请问如何解释下图里的内容?")
print(content)

这幅图片显示的是一份化学学科的笔记，内容重点介绍了溶液的酸碱性判定，使用指示剂的变色范围，以及溶液中离子的定性分析。

1. **做酸碱滴定时的注意事项**：原文提到在做酸碱滴定时，如果在溶液中加入了过多的指示剂，可能会影响终点的判断，因为过多的指示剂可以分解生成色素，从而影响颜色的变化，使得判定终点时出现误差。

2. **指示剂的变色范围**：笔记列出了常见的几种指示剂（如酚酞、甲基橙等）的变色pH范围，这对于选择合适的指示剂进行滴定实验时非常重要，特别是在滴定曲线的制备以及滴定终点的准确识别。

3. **离子的定性分析**：文中还涉及了一些关于离子（如铜离子等）在溶液中的定性分析方法，如铜离子在硫氢化钠NaSH存在时会生成暗蓝色沉淀，这可以用于实验中的铜离子的检测。

4. **讨论及原理解析**：笔记中还包括了关于酸碱性质、离子浓度等的原理解析，以及实验误差的来源和避免方法，有助于学生在实验操作中更准确地掌握技术和理论知识。

总之，这些笔记非常详细地解释了化学实验中的一些基本概念和技巧，特别适合中学生在学习化学实验时作为参考和复习使用。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [16]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这幅图片显示的是一份化学学科的笔记，内容重点介绍了溶液的酸碱性判定，使用指示剂的变色范围，以及溶液中离子的定性分析。

1. **做酸碱滴定时的注意事项**：原文提到在做酸碱滴定时，如果在溶液中加入了过多的指示剂，可能会影响终点的判断，因为过多的指示剂可以分解生成色素，从而影响颜色的变化，使得判定终点时出现误差。

2. **指示剂的变色范围**：笔记列出了常见的几种指示剂（如酚酞、甲基橙等）的变色pH范围，这对于选择合适的指示剂进行滴定实验时非常重要，特别是在滴定曲线的制备以及滴定终点的准确识别。

3. **离子的定性分析**：文中还涉及了一些关于离子（如铜离子等）在溶液中的定性分析方法，如铜离子在硫氢化钠NaSH存在时会生成暗蓝色沉淀，这可以用于实验中的铜离子的检测。

4. **讨论及原理解析**：笔记中还包括了关于酸碱性质、离子浓度等的原理解析，以及实验误差的来源和避免方法，有助于学生在实验操作中更准确地掌握技术和理论知识。

总之，这些笔记非常详细地解释了化学实验中的一些基本概念和技巧，特别适合中学生在学习化学实验时作为参考和复习使用。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。